In [43]:
#imports
import requests
import pandas as pd
import datetime as dt
from google.cloud import bigquery
from google.cloud import storage
from shapely.geometry  import shape, Point
from geopy.geocoders import Nominatim
import numpy as np
import meteostat as ms
import os
import io
import json

In [44]:
CREDS = "C:/Users/tkkim/gcp_keys/capstone-team51-366963bafc54.json"
storage_client = storage.Client.from_service_account_json(json_credentials_path=CREDS,project='capstone-team51')
bq_client = bigquery.Client.from_service_account_json(json_credentials_path=CREDS,project='capstone-team51')
#client = bigquery.Client(project='capstone-team51')


In [45]:
geolocator = Nominatim(user_agent="test_tk")

In [46]:
bucket = storage_client.get_bucket('capstone-team51-data')

In [47]:
def get_commarea_env(dataframe, commarea):
    dataframe['community_area']=''
    for row in dataframe.index:
        point = Point(dataframe['location.coordinates'][row][0],dataframe['location.coordinates'][row][1])
        for feature in commarea['features']:
            polygon = shape(feature['geometry'])
            if polygon.contains(point):
                dataframe['community_area'][row] = feature['properties']['area_numbe']

In [48]:
def getpoint(address):
    loc = geolocator.geocode(address)
    latlong = [loc.latitude, loc.longitude]
    point = Point(latlong[1], latlong[0])   
    return point

In [49]:
def findarea(point, commarea):    
    for feature in commarea['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point):
           return feature['properties']['area_numbe']
        else:
            continue
        

In [50]:
offenders_df = pd.read_csv("C:/Users/tkkim/Desktop/498_winter_group_bfmnt/illinois_state_offenders.csv",quotechar='\x07',index_col=False)

In [51]:
offenders_df.head()

,Last Name,First Name,Middle Name,Address,City,State,Zip Code,Residence County,Height,Weight,Race,Gender,Date of Birth,Status,Classification,Conviction State or IL County,Age of Victim,Age of Offender at Time of Offense,Crimes:,Unnamed: 19
0,WESTON,ROBERT,NaN,out of state,NaN,NaN,NaN,Out of State,"5'11""",188 lbs.,W,M,2/14/1973,Compliant,Non-verified Adult Sex Offender,Cook,NaN,NaN,0012293 - CRIMINAL SEXUAL ABUSE FORCE,NaN
1,MATHIS,JIMMY,R,out of state,NaN,NaN,NaN,Out of State,"5'06""",150 lbs.,W,M,8/11/1949,Compliant,Sexual Predator,Lake,NaN,NaN,0999100 - AGGRAVATED CRIMINAL SEXUAL ABUSE/WEAPON,0999400 - AGGRAVATED CRIMINAL SEXUAL ABUSE/VIC...
2,BUTLER,ANTONIO,C,UNK ADDRESS UNKNOWN,NaN,IL,NaN,NaN,"5'10""",320 lbs.,B,M,9/26/1974,Location Unknown,Sexual Predator,Cook,14.0,25.0,0999200 - AGGRAVATED CRIMINAL SEX ABUSE/BODILY...,NaN
3,EDWARDS,EARL,R,Department of Corrections,NaN,NaN,NaN,NaN,"6'01""",250 lbs.,B,M,6/15/1970,Compliant,Sexual Predator,Cook,0.0,21.0,0996100 - AGGRAVATED CRIMINAL SEXUAL ASSAULT,0996100 - AGGRAVATED CRIMINAL SEXUAL ASSAULT
4,LOPEZ,JESUS,NaN,out of state,NaN,NaN,NaN,Out of State,"5'06""",180 lbs.,U,M,12/28/1973,Compliant,Sexual Predator,Cook,9.0,23.0,0999200 - AGGRAVATED CRIMINAL SEX ABUSE/BODILY...,NaN


In [52]:
droplist = ['out of state','Department of Corrections', 'Homeless', 'unknown', 'UNK ADDRESS UNKNOWN ']

In [53]:
cleaned_offenders = offenders_df[offenders_df['City'].fillna('unknown').str.contains('CHICAGO')]

In [54]:
cleaned_offenders = cleaned_offenders[~cleaned_offenders['Address'].fillna('unknown').isin(droplist)]

In [55]:
cleaned_offenders = cleaned_offenders[~cleaned_offenders['Address'].str.contains('REFUS')]

In [56]:
cleaned_offenders.reset_index(drop=True, inplace=True)

In [57]:
cleaned_offenders.head()

,Last Name,First Name,Middle Name,Address,City,State,Zip Code,Residence County,Height,Weight,Race,Gender,Date of Birth,Status,Classification,Conviction State or IL County,Age of Victim,Age of Offender at Time of Offense,Crimes:,Unnamed: 19
0,CROCKETT,DONALD,NaN,417 W WINNECONNA PKWY,CHICAGO,IL,60620,Cook,"5'11""",175 lbs.,B,M,3/13/1956,Compliant,Sexual Predator,Cook,22.0,24.0,0790000 - RAPE,0790000 - RAPE
1,ROGERS,ERIC,C,9943 S S THROOP ST,CHICAGO,IL,60643,Cook,"5'11""",190 lbs.,B,M,7/22/1961,Non-Compliant,Non-verified Adult Sex Offender,US,21.0,30.0,0013639 - FAILURE TO REPORT A CHANGE OF ADDRES...,0996300 - AGGRAVATED CRIMINAL SEXUAL ASSAULT
2,LUCAS,WILLIAM,NaN,5523 S NEVA AVE,CHICAGO,IL,60638,Cook,"5'10""",190 lbs.,U,M,1/29/1956,Compliant,Sexual Predator,Cook,14.0,43.0,0011423 - CHILD PORNOGRAPHY/FILM/PHOTOS,0010314 - INDECENT SOLICITATION OF AN ADULT
3,TAYLOR,COREY,NaN,1740 W TAYLOR ST,CHICAGO,IL,60612,Cook,"5'05""",185 lbs.,B,M,2/23/1969,Non-Compliant,Sexual Predator,Cook,16.0,35.0,ISP9999 - FELONY CONVICTION AFTER 7/1/2011,0996300 - AGGRAVATED CRIMINAL SEXUAL ASSAULT
4,MANCEDO,CELESTE,NaN,10017 S AVENUE N,CHICAGO,IL,60617,Cook,"5'02""",150 lbs.,U,F,2/8/1982,Compliant,Non-verified Child Sex Offender,Cook,17.0,17.0,0012293 - CRIMINAL SEXUAL ABUSE FORCE,0011141 - VIOLATE SEX OFFENDER REGISTRATION ACT


In [85]:
cleaned_offenders['City'] = 'CHICAGO'

In [76]:
point = getpoint('9943 S S THROOP ST CHICAGO')

In [80]:
for feature in commarea['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point):
            print(feature['properties']['area_numbe'])

73


In [75]:
x.head()

,Address,City,fixed_address,community_area
0,417 W WINNECONNA PKWY,CHICAGO,417 W WINNECONNA PKWY CHICAGO,no match to osm
1,9943 S S THROOP ST,CHICAGO,9943 S S THROOP ST CHICAGO,no match to osm
2,5523 S NEVA AVE,CHICAGO,5523 S NEVA AVE CHICAGO,no match to osm
3,1740 W TAYLOR ST,CHICAGO,1740 W TAYLOR ST CHICAGO,no match to osm
4,10017 S AVENUE N,CHICAGO,10017 S AVENUE N CHICAGO,no match to osm


In [82]:
x = get_commarea_offenders(cleaned_offenders, commarea)

0.0% done
0.03% done
0.06999999999999999% done
0.1% done
0.13% done
0.16% done
0.2% done
0.22999999999999998% done
0.26% done
0.3% done
0.33% done
0.36% done
0.38999999999999996% done
0.43% done
0.45999999999999996% done
0.49% done
0.53% done
0.5599999999999999% done
0.59% done
0.62% done
0.66% done
0.69% done
0.72% done
0.76% done
0.79% done
0.8200000000000001% done
0.8500000000000001% done
0.89% done
0.9199999999999999% done
0.95% done
0.98% done
1.02% done
1.05% done
1.08% done
1.1199999999999999% done
1.15% done
1.18% done
1.21% done
1.25% done
1.28% done
1.31% done
1.35% done
1.38% done
1.41% done
1.44% done
1.48% done
1.51% done
1.54% done
1.58% done
1.6099999999999999% done
1.6400000000000001% done
1.67% done
1.71% done
1.7399999999999998% done
1.77% done
1.81% done
1.8399999999999999% done
1.87% done
1.9% done
1.94% done
1.97% done
2.0% done
2.04% done
2.07% done
2.1% done
2.13% done
2.17% done
2.1999999999999997% done
2.23% done
2.27% done
2.3% done
2.33% done
2.36% done
2.4% 

In [88]:
x.iloc[3044]

Address                   8119 W S COTTAGE GROVE 
City                                      CHICAGO
fixed_address     8119 W S COTTAGE GROVE  CHICAGO
community_area                    no match to osm
Name: 3044, dtype: object

In [89]:
x.to_csv('./partially_cleaned_offenders.csv', index=False)

In [87]:
x[x['community_area']=='no match to osm'][['Address']]

,Address
4,10017 S AVENUE N
6,5806 S BISHOP ST 1
7,6315 W W GUNNISON HSE HSE
12,3411 W MONROE ST 2R
13,201 W NORTH AVE
...,...
3036,4559 S HALSTED ST 3RD FL
3040,12141 W S GREEN STREE 1
3041,2500 S THROOP ST 18
3043,3351 W OHIO ST 3E


In [81]:
def get_commarea_offenders(dataframe, commarea):
    newdataframe = dataframe[['Address', 'City']].copy()
    newdataframe['fixed_address'] = newdataframe['Address'] + ' ' + newdataframe['City']
    newdataframe['community_area']=''
    i=0
    for row in newdataframe.index:
        try:
            point = getpoint(newdataframe['fixed_address'][row])
            newdataframe.loc[row,'community_area'] = findarea(point, commarea)
        except:
            newdataframe.loc[row,'community_area'] = 'no match to osm'
        print(f'{round((i/newdataframe.shape[0]),4)*100}% done')
        i+=1
    return newdataframe


In [20]:
def load_df_gcs(prefix, filename):
    df = pd.DataFrame()
    for blob in storage_client.list_blobs('capstone-team51-data', prefix=prefix, delimiter='/'):
        #print(blob.name)
        if filename in blob.name:
            #print(blob.name)
            data = blob.download_as_bytes()
            smalldf = pd.read_csv(io.BytesIO(data))
            #print(smalldf)
            df = pd.concat([df, smalldf])
    return df

In [99]:
base_df = load_df_gcs('raw_crimes/', 'chunk')
three11_df = load_df_gcs('raw_311/', 'chunk')
env_df = load_df_gcs('raw_environmental/', 'chunk')
weather_df = load_df_gcs('raw_weather/', 'data')

In [88]:
#stations_blob = bucket.blob('supporting_data/Police_Stations_20240120.csv').download_as_bytes()
stations_df = load_df_gcs('supporting/', 'Police_Stations_20240120.csv')

In [21]:
commarea = json.loads(bucket.blob('supporting/geojsons/Boundaries - Community Areas (current).geojson').download_as_string())

In [100]:
env_df.head()

,complaint_id,complaint_type,address,street_number,direction,street_name,street_type,inspector,complaint_date,inspection_log,data_source,modified_date,latitude,longitude,location.type,location.coordinates,complaint_detail
0,601485709,CONSTRUCTION AND DEMOLITION,1934 N SEMINARY AVE,1934,N,SEMINARY,AVE,379542,2024-01-02T00:00:00.000,[INSPECTION LOG #: 21281737 02-JAN-24 15:00:00...,DEPT. OF PUBLIC HEALTH,2024-01-08T00:00:00.000,41.917036,-87.655529,Point,"[-87.65552929240022, 41.917035884002566]",NaN
1,601485743,AIR POLLUTION WORK ORDER,6633 S MOZART ST,6633,S,MOZART,ST,249349,2024-01-02T00:00:00.000,[INSPECTION LOG #: 21262766 ] ON 1/2/2024 CDP...,DEPT. OF PUBLIC HEALTH,2024-01-03T00:00:00.000,41.772635,-87.694304,Point,"[-87.6943035754064, 41.772635074254794]",NaN
2,601485705,NOISE COMPLAINT,209 E OHIO ST,209,E,OHIO,ST,437489,2024-01-02T00:00:00.000,[INSPECTION LOG #: 21271208 02-JAN-24 12:25:00...,DEPT. OF PUBLIC HEALTH,2024-01-04T00:00:00.000,41.892456,-87.622264,Point,"[-87.62226362586607, 41.89245644738031]",NaN
3,601485714,CONSTRUCTION AND DEMOLITION,6458 S BISHOP ST,6458,S,BISHOP,ST,8386,2024-01-02T00:00:00.000,[INSPECTION LOG #: 21259084 02-JAN-24 14:10:00...,DEPT. OF PUBLIC HEALTH,2024-01-02T00:00:00.000,41.775973,-87.660622,Point,"[-87.66062216016986, 41.77597289860283]",NaN
4,601485703,CONSTRUCTION AND DEMOLITION,6 S LAFLIN ST,6,S,LAFLIN,ST,3308,2024-01-02T00:00:00.000,[INSPECTION LOG #: 21279463 02-JAN-24 11:15:00...,DEPT. OF PUBLIC HEALTH,2024-01-05T00:00:00.000,41.881322,-87.664561,Point,"[-87.66456137666425, 41.88132153435288]",NaN


In [93]:
io.BytesIO(geojson)

In [89]:
stations_df.head()

,DISTRICT,DISTRICT NAME,ADDRESS,CITY,STATE,ZIP,WEBSITE,PHONE,FAX,TTY,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION
0,Headquarters,Headquarters,3510 S Michigan Ave,Chicago,IL,60653,http://home.chicagopolice.org,NaN,NaN,NaN,1177731.401,1881697.404,41.830702,-87.623395,"(41.8307016873, -87.6233953459)"
1,18,Near North,1160 N Larrabee St,Chicago,IL,60610,http://home.chicagopolice.org/community/distri...,312-742-5870,312-742-5771,312-742-5773,1172080.029,1908086.527,41.903242,-87.643352,"(41.9032416531, -87.6433521393)"
2,19,Town Hall,850 W Addison St,Chicago,IL,60613,http://home.chicagopolice.org/community/distri...,312-744-8320,312-744-4481,312-744-8011,1169730.744,1924160.317,41.947400,-87.651512,"(41.9474004564, -87.651512018)"
3,20,Lincoln,5400 N Lincoln Ave,Chicago,IL,60625,http://home.chicagopolice.org/community/distri...,312-742-8714,312-742-8803,312-742-8841,1158399.146,1935788.826,41.979550,-87.692845,"(41.9795495131, -87.6928445094)"
4,22,Morgan Park,1900 W Monterey Ave,Chicago,IL,60643,http://home.chicagopolice.org/community/distri...,312-745-0710,312-745-0814,312-745-0569,1165825.476,1830851.333,41.691435,-87.668520,"(41.6914347795, -87.6685203937)"


In [68]:
prefix_file_dict = {'raw_crimes/': 'chunk', 
                    'raw_311/': 'chunk', 
                    'raw_environmental/': 'chunk', 
                    'raw_weather/': 'data'}

In [77]:
df_holder = []
for key, value in prefix_file_dict.items():
    smallholder = {}
    df = load_df_gcs(key, value)
    smallholder['name'] = key
    smallholder['df'] = df
    df_holder.append(smallholder)

raw_crimes/
raw_311/
raw_environmental/
raw_weather/


In [ ]:
base_df = pd.DataFrame()

In [78]:
for pair in df_holder:
    print(pair['name'])
    print(pair['df'].head())

raw_crimes/
         id case_number                     date                 block  iucr  \
0  13325552    JH100861  2023-12-31T00:00:00.000     019XX W CERMAK RD  0910   
1  13324259    JG560685  2023-12-31T00:00:00.000  094XX S ELIZABETH ST  0560   
2  13327627    JH103322  2023-12-31T00:00:00.000    0000X W HUBBARD ST  1310   
3  13324407    JG560896  2023-12-31T00:00:00.000  121XX S MICHIGAN AVE  0920   
4  13326606    JH102040  2023-12-31T00:00:00.000  006XX N LOCKWOOD AVE  1320   

          primary_type           description             location_description  \
0  MOTOR VEHICLE THEFT            AUTOMOBILE                      GAS STATION   
1              ASSAULT                SIMPLE                        RESIDENCE   
2      CRIMINAL DAMAGE           TO PROPERTY                        APARTMENT   
3  MOTOR VEHICLE THEFT  ATTEMPT - AUTOMOBILE                           STREET   
4      CRIMINAL DAMAGE            TO VEHICLE  RESIDENCE - YARD (FRONT / BACK)   

   arrest  domestic 

In [73]:
for key, value in df_holder.items():
    print(value)
    

raw_weather/
          time  tavg  tmin  tmax  prcp   snow   wdir  wspd  wpgt    pres  \
0   2024-01-01   0.1  -1.7   1.1   0.0    0.0  307.0  19.3   NaN  1025.8   
1   2024-01-02   0.7  -2.2   5.0   0.0    0.0  251.0  21.5   NaN  1022.1   
2   2024-01-03   1.4   0.0   2.8   0.0    0.0  304.0  12.7   NaN  1019.4   
3   2024-01-04   0.8  -2.2   3.3   0.0    0.0  324.0  10.2   NaN  1027.3   
4   2024-01-05   0.5  -2.2   2.8   0.0    0.0  167.0  10.7   NaN  1022.9   
5   2024-01-06   0.7  -0.6   1.7   5.2   25.0  176.0   8.4   NaN  1009.0   
6   2024-01-07   1.0   0.6   1.7   0.2   25.0  271.0  14.2   NaN  1015.1   
7   2024-01-08   1.8   0.0   3.3   0.8   30.0  127.0  14.3   NaN  1022.2   
8   2024-01-09   1.6   1.1   2.8  20.3   30.0   45.0  19.1   NaN   990.7   
9   2024-01-10   0.6  -0.6   1.7   0.0   25.0  245.0  20.2   NaN  1000.7   
10  2024-01-11   0.0  -2.2   1.7   0.4   25.0  233.0  14.6   NaN  1009.4   
11  2024-01-12   1.7  -0.6   3.3  15.8   50.0  107.0  23.2   NaN   994.9   

In [65]:
x = load_df_gcs('raw_weather/', 'data')

In [66]:
x.head()

,time,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,tempmax_f,tempmin_f,tempavg_f
0,2024-01-01,0.1,-1.7,1.1,0.0,0.0,307.0,19.3,NaN,1025.8,NaN,33.98,28.94,32.18
1,2024-01-02,0.7,-2.2,5.0,0.0,0.0,251.0,21.5,NaN,1022.1,NaN,41.00,28.04,33.26
2,2024-01-03,1.4,0.0,2.8,0.0,0.0,304.0,12.7,NaN,1019.4,NaN,37.04,32.00,34.52
3,2024-01-04,0.8,-2.2,3.3,0.0,0.0,324.0,10.2,NaN,1027.3,NaN,37.94,28.04,33.44
4,2024-01-05,0.5,-2.2,2.8,0.0,0.0,167.0,10.7,NaN,1022.9,NaN,37.04,28.04,32.90


In [41]:
for tho in storage_client.list_blobs('capstone-team51-data', prefix='raw_311/', delimiter='/'):
    print(tho.name)

raw_311/
raw_311/20240204_311_chunk_0.csv


In [36]:
for x in raw311_blobs:
    print(x.name)

In [19]:
x

NameError: name 'x' is not defined

In [11]:
for x in storage_client.list_blobs('capstone-team51-data', prefix='raw311'):
  print(x)


In [5]:
bucket_311 = buc

ImportError: Missing optional dependency 'fsspec'.  Use pip or conda to install fsspec.